In [1]:
# | default_exp nets/vit_3d

# Imports

In [2]:
# | export

import numpy as np
import torch
from einops import rearrange, repeat
from huggingface_hub import PyTorchModelHubMixin
from torch import nn

# Architecture

### Basic Layers

In [3]:
# | export


def get_coords_grid(grid_size):
    d, h, w = grid_size

    grid_d = torch.arange(d, dtype=torch.int32)
    grid_h = torch.arange(h, dtype=torch.int32)
    grid_w = torch.arange(w, dtype=torch.int32)

    grid = torch.meshgrid(grid_w, grid_h, grid_d, indexing="ij")
    grid = torch.stack(grid, axis=0)
    # (3, d, h, w)

    return grid

In [4]:
# | export


class ViT3DMHA(nn.Module):  # Multi-head-attention
    def __init__(
        self,
        dim,
        num_heads,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
    ):
        super().__init__()

        assert dim % num_heads == 0, "dimension must be divisible by number of heads"

        self.dim = dim
        self.num_heads = num_heads

        self.per_head_dim = int(dim // num_heads)

        self.W_q = nn.Linear(dim, dim)
        self.W_k = nn.Linear(dim, dim)
        self.W_v = nn.Linear(dim, dim)
        self.attn_drop = nn.Dropout(attn_drop_prob)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_prob)

    def forward(self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor):
        # q: (b, num_tokens_of_q, dim)
        # k: (b, num_tokens_of_k, dim)
        # v: (b, num_tokens_of_v, dim)

        query = rearrange(
            self.W_q(q), "b num_tokens (num_heads d) -> b num_heads num_tokens d", num_heads=self.num_heads
        )
        key = rearrange(self.W_k(k), "b num_tokens (num_heads d) -> b num_heads num_tokens d", num_heads=self.num_heads)
        value = rearrange(
            self.W_v(v), "b num_tokens (num_heads d) -> b num_heads num_tokens d", num_heads=self.num_heads
        )
        # Each is (b, num_heads, num_tokens_in_the_tensor, per_head_dim)

        attention_scores = query @ rearrange(key, "b num_heads num_tokens d -> b num_heads d num_tokens")
        attention_scores = attention_scores / (self.per_head_dim**0.5)
        # (b, num_heads, num_tokens_in_q, num_tokens_in_kv)

        attention_probs = nn.functional.softmax(attention_scores, dim=-1)
        attention_probs = self.attn_drop(attention_probs)
        # (b, num_heads, num_tokens_in_q, num_tokens_in_kv)

        context = attention_probs @ value
        # (b, num_heads, num_tokens_in_q, per_head_dim)
        context = rearrange(context, "b num_heads num_tokens d -> b num_tokens (num_heads d)")
        # (b, num_tokens_of_q, dim)

        context = self.proj(context)
        context = self.proj_drop(context)
        # (b, num_tokens_of_q, dim)

        return context

In [5]:
# | export


class ViT3DMHSA(ViT3DMHA):  # Multi head self attention
    def __init__(
        self,
        dim,
        num_heads,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
    ):
        super().__init__(
            dim,
            num_heads,
            attn_drop_prob,
            proj_drop_prob,
        )

    def forward(self, qkv: torch.Tensor):
        return super().forward(qkv, qkv, qkv)

In [6]:
test = ViT3DMHSA(54, 6)
display(test)
display(test(torch.randn(2, 64, 54)).shape)


ViT3DMHSA(
  (W_q): Linear(in_features=54, out_features=54, bias=True)
  (W_k): Linear(in_features=54, out_features=54, bias=True)
  (W_v): Linear(in_features=54, out_features=54, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=54, out_features=54, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

torch.Size([2, 64, 54])

In [7]:
# | export


class ViT3DMHCA(ViT3DMHA):  # Multi head cross attention
    def __init__(
        self,
        dim,
        num_heads,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
    ):
        super().__init__(
            dim,
            num_heads,
            attn_drop_prob,
            proj_drop_prob,
        )

    def forward(self, q: torch.Tensor, kv: torch.Tensor):
        return super().forward(q, kv, kv)

In [8]:
test = ViT3DMHCA(54, 6)
display(test)
display(test(torch.randn(2, 64, 54), torch.randn(2, 64, 54)).shape)


ViT3DMHCA(
  (W_q): Linear(in_features=54, out_features=54, bias=True)
  (W_k): Linear(in_features=54, out_features=54, bias=True)
  (W_v): Linear(in_features=54, out_features=54, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=54, out_features=54, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

torch.Size([2, 64, 54])

In [9]:
test = ViT3DMHCA(54, 6)
display(test)
display(test(torch.randn(2, 1, 54), torch.randn(2, 64, 54)).shape)


ViT3DMHCA(
  (W_q): Linear(in_features=54, out_features=54, bias=True)
  (W_k): Linear(in_features=54, out_features=54, bias=True)
  (W_v): Linear(in_features=54, out_features=54, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=54, out_features=54, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

torch.Size([2, 1, 54])

In [10]:
# | export


class ViT3DLayerMLP(nn.Module):
    def __init__(self, dim, intermediate_ratio, dropout_prob=0.0):
        super().__init__()
        self.dense1 = nn.Linear(dim, dim * intermediate_ratio)
        self.act = nn.GELU()
        self.dense2 = nn.Linear(dim * intermediate_ratio, dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_tokens, dim)
        hidden_states = self.dense1(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.dense2(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states

In [11]:
test = ViT3DLayerMLP(64, 256)
display(test)
display(test(torch.randn(2, 64, 64)).shape)


ViT3DLayerMLP(
  (dense1): Linear(in_features=64, out_features=16384, bias=True)
  (act): GELU(approximate='none')
  (dense2): Linear(in_features=16384, out_features=64, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

torch.Size([2, 64, 64])

### Encoder

In [12]:
# | export


class ViT3DEncoderLayer(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        intermediate_ratio,
        layer_norm_eps,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__()

        self.mhsa = ViT3DMHSA(dim, num_heads, attn_drop_prob, proj_drop_prob)
        self.layernorm1 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mlp = ViT3DLayerMLP(dim, intermediate_ratio, mlp_drop_prob)
        self.layernorm2 = nn.LayerNorm(dim, eps=layer_norm_eps)

    def forward(self, qkv: torch.Tensor):
        """Note: This uses post-normalization instead of pre-normalization. This is inspired by SwinV2"""
        # qkv: (b, num_tokens, dim)

        res_connection1 = qkv
        # (b, num_tokens, dim)

        hidden_states = self.mhsa(qkv)
        hidden_states = self.layernorm1(hidden_states)
        # (b, num_tokens, dim)

        res_connection2 = hidden_states + res_connection1
        # (b, num_tokens, dim)

        hidden_states = self.mlp(res_connection2)
        hidden_states = self.layernorm2(hidden_states)
        # (b, num_tokens, dim)

        hidden_states = hidden_states + res_connection2
        # (b, num_tokens, dim)

        return hidden_states

In [13]:
test = ViT3DEncoderLayer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 64, 52)).shape)


ViT3DEncoderLayer(
  (mhsa): ViT3DMHSA(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): ViT3DLayerMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
)

torch.Size([2, 64, 52])

In [14]:
# | export


class ViT3DEncoder(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config):
        super().__init__()

        self.layers = nn.ModuleList(
            [
                ViT3DEncoderLayer(
                    config["dim"],
                    config["num_heads"],
                    config["intermediate_ratio"],
                    config["layer_norm_eps"],
                    config["attn_drop_prob"],
                    config["proj_drop_prob"],
                    config["mlp_drop_prob"],
                )
                for _ in range(config["encoder_depth"])
            ]
        )

    def forward(self, embeddings: torch.Tensor):
        # hidden_states: (b, num_tokens, dim)

        layer_outputs = []
        for encoder_layer in self.layers:
            embeddings = encoder_layer(embeddings)
            # (b, num_tokens, dim)

            layer_outputs.append(embeddings)

        return embeddings, layer_outputs

In [15]:
test_config = {
    "dim": 54,
    "num_heads": 6,
    "intermediate_ratio": 2,
    "layer_norm_eps": 1e-6,
    "attn_drop_prob": 0.0,
    "proj_drop_prob": 0.0,
    "mlp_drop_prob": 0.0,
    "encoder_depth": 5,
}

test = ViT3DEncoder(test_config)
display(test)
o = test(torch.randn(2, 64, 54))
display((o[0].shape, [x.shape for x in o[1]]))


ViT3DEncoder(
  (layers): ModuleList(
    (0-4): 5 x ViT3DEncoderLayer(
      (mhsa): ViT3DMHSA(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
      (mlp): ViT3DLayerMLP(
        (dense1): Linear(in_features=54, out_features=108, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=108, out_features=54, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
    )
  )
)


(
    torch.Size([2, 64, 54]),
    [
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54])
    ]
)

### Decoder

In [16]:
# | export


class ViT3DDecoderLayer(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        intermediate_ratio,
        layer_norm_eps,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__()

        self.mhsa = ViT3DMHSA(dim, num_heads, attn_drop_prob, proj_drop_prob)
        self.layernorm1 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mhca = ViT3DMHCA(dim, num_heads, attn_drop_prob, proj_drop_prob)
        self.layernorm2 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mlp = ViT3DLayerMLP(dim, intermediate_ratio, mlp_drop_prob)
        self.layernorm3 = nn.LayerNorm(dim, eps=layer_norm_eps)

    def forward(self, q: torch.Tensor, kv: torch.Tensor):  # This uses post-normalization
        # q: (b, num_tokens_in_q, dim)
        # kv: (b, num_tokens_in_kv, dim)

        res_connection1 = q
        # (b, num_tokens_in_q, dim)

        hidden_states_q = self.mhsa(q)
        hidden_states_q = self.layernorm1(hidden_states_q)
        # (b, num_tokens_in_q, dim)

        res_connection2 = hidden_states_q + res_connection1
        # (b, num_tokens_in_q, dim)

        hidden_states = self.mhca(res_connection2, kv)
        hidden_states = self.layernorm2(hidden_states)
        # (b, num_tokens_in_q, dim)

        res_connection3 = hidden_states + res_connection2
        # (b, num_tokens_in_q, dim)

        hidden_states = self.mlp(res_connection3)
        hidden_states = self.layernorm3(hidden_states)
        # (b, num_tokens_in_q, dim)

        hidden_states = hidden_states + res_connection3
        # (b, num_tokens_in_q, dim)

        return hidden_states

In [17]:
test = ViT3DDecoderLayer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 64, 52), torch.randn(2, 64, 52)).shape)


ViT3DDecoderLayer(
  (mhsa): ViT3DMHSA(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mhca): ViT3DMHCA(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): ViT3DLayerMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(

torch.Size([2, 64, 52])

In [18]:
test = ViT3DDecoderLayer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 3, 52), torch.randn(2, 64, 52)).shape)


ViT3DDecoderLayer(
  (mhsa): ViT3DMHSA(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mhca): ViT3DMHCA(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): ViT3DLayerMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(

torch.Size([2, 3, 52])

In [19]:
# | export


class ViT3DDecoder(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config):
        super().__init__()

        self.layers = nn.ModuleList(
            [
                ViT3DDecoderLayer(
                    config["dim"],
                    config["num_heads"],
                    config["intermediate_ratio"],
                    config["layer_norm_eps"],
                    config["attn_drop_prob"],
                    config["proj_drop_prob"],
                    config["mlp_drop_prob"],
                )
                for _ in range(config["decoder_depth"])
            ]
        )

    def forward(self, q: torch.Tensor, kv: torch.Tensor):
        # q: (b, num_q_tokens, dim)
        # kv: (b, num_kv_tokens, dim)

        embeddings = q

        layer_outputs = []
        for decoder_layer in self.layers:
            embeddings = decoder_layer(embeddings, kv)
            # (b, num_q_tokens, dim)

            layer_outputs.append(embeddings)

        return embeddings, layer_outputs

In [20]:
test_config = {
    "dim": 54,
    "num_heads": 6,
    "intermediate_ratio": 2,
    "layer_norm_eps": 1e-6,
    "attn_drop_prob": 0.0,
    "proj_drop_prob": 0.0,
    "mlp_drop_prob": 0.0,
    "decoder_depth": 5,
}

test = ViT3DDecoder(test_config)
display(test)
o = test(torch.randn(2, 64, 54), torch.randn(2, 128, 54))
display((o[0].shape, [x.shape for x in o[1]]))


ViT3DDecoder(
  (layers): ModuleList(
    (0-4): 5 x ViT3DDecoderLayer(
      (mhsa): ViT3DMHSA(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
      (mhca): ViT3DMHCA(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((54,), eps=1e-06, elemen


(
    torch.Size([2, 64, 54]),
    [
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54])
    ]
)

# Embeddings

### Patch embeddings

In [21]:
# | export


class ViT3DPatchEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()

        patch_size = config["patch_size"]
        num_channels = config["in_channels"]
        dim = config["dim"]

        self.patch_embeddings = nn.Conv3d(
            in_channels=num_channels,
            out_channels=dim,
            kernel_size=patch_size,
            stride=patch_size,
        )

    def forward(self, pixel_values: torch.Tensor):
        # pixel_values: (b, c, z, y, x)

        embeddings = self.patch_embeddings(pixel_values)
        # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

        return embeddings

In [22]:
test_config = {
    "patch_size": (1, 8, 8),
    "in_channels": 1,
    "dim": 12,
}

test = ViT3DPatchEmbeddings(test_config)
display(test)
o = test(torch.randn(2, 1, 32, 512, 512))
display(o.shape)


ViT3DPatchEmbeddings(
  (patch_embeddings): Conv3d(1, 12, kernel_size=(1, 8, 8), stride=(1, 8, 8))
)

torch.Size([2, 12, 32, 64, 64])

### Position embeddings

In [23]:
# | export


def get_3d_position_embeddings(embedding_size, grid_size, patch_size=(1, 1, 1)):
    if embedding_size % 6 != 0:
        raise ValueError("embed_dim must be divisible by 6")

    grid = get_coords_grid(grid_size)
    # (3, d, h, w)

    grid = rearrange(grid, "x d h w -> x 1 d h w")
    # (3, 1, d, h, w)

    omega = torch.arange(embedding_size // 6, dtype=torch.float32)
    omega /= embedding_size / 6.0
    omega = 1.0 / 10000**omega
    # (d // 6)

    patch_multiplier = torch.Tensor(patch_size) / min(patch_size)

    position_embeddings = []
    for i, grid_subset in enumerate(grid):
        grid_subset = grid_subset.reshape(-1)
        out = torch.einsum("m,d->md", grid_subset, omega)

        emb_sin = torch.sin(out)
        emb_cos = torch.cos(out)

        emb = torch.cat([emb_sin, emb_cos], axis=1) * patch_multiplier[i]
        position_embeddings.append(emb)

    position_embeddings = torch.cat(position_embeddings, axis=1)
    # (embedding_size, d * h * w)
    d, h, w = grid_size
    position_embeddings = rearrange(position_embeddings, "(d h w) e -> 1 e d h w", d=d, h=h, w=w)
    # (1, embedding_size, d, h, w)

    return position_embeddings

In [24]:
# | export


def embed_spacings_in_position_embeddings(embeddings: torch.Tensor, spacings: torch.Tensor):
    assert spacings.ndim == 2, "Please provide spacing information for each batch element"
    _, embedding_size, _, _, _ = embeddings.shape
    assert embedding_size % 3 == 0, "To embed spacing info, the embedding size must be divisible by 3"
    embeddings = embeddings * repeat(spacings, f"B S -> B (S {int(embedding_size / 3)}) 1 1 1", S=3)

    return embeddings

In [25]:
# | export


class ViT3DEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config

        dim = config["dim"]

        self.patch_embeddings = ViT3DPatchEmbeddings(config)
        self.layer_norm = nn.LayerNorm(dim)

        self.absolute_position_embeddings = None
        if config["use_absolute_position_embeddings"]:
            grid_size = (
                config["image_size"][0] // config["patch_size"][0],
                config["image_size"][1] // config["patch_size"][1],
                config["image_size"][2] // config["patch_size"][2],
            )
            if config["learnable_absolute_position_embeddings"]:
                self.absolute_position_embeddings = nn.Parameter(
                    torch.randn(1, dim, grid_size[0], grid_size[1], grid_size[2])
                )
            else:
                self.absolute_position_embeddings = get_3d_position_embeddings(dim, grid_size, config["patch_size"])

    def forward(
        self,
        pixel_values: torch.Tensor,
        spacings: torch.Tensor,
        mask_patches: torch.Tensor = None,
        mask_token: torch.Tensor = None,
    ):
        # pixel_values: (b, c, z, y, x)

        embeddings = self.patch_embeddings(pixel_values)
        # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)
        embeddings = rearrange(embeddings, "b d nz ny nx -> b nz ny nx d")
        embeddings = self.layer_norm(embeddings)
        embeddings = rearrange(embeddings, "b nz ny nx d -> b d nz ny nx")
        # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

        if mask_patches is not None:
            # mask_patches (binary mask): (b, num_tokens_z, num_tokens_y, num_tokens_x)
            # mask_token: (1, dim, 1, 1, 1)
            mask_patches = repeat(mask_patches, "b z y x -> b d z y x", d=embeddings.shape[1])
            embeddings = (embeddings * (1 - mask_patches)) + (mask_patches * mask_token)

        if self.absolute_position_embeddings is not None:
            absolute_position_embeddings = self.absolute_position_embeddings.to(embeddings.device)
            # (1, dim, num_tokens_z, num_tokens_y, num_tokens_x)
            if self.config["embed_spacing_info"]:
                absolute_position_embeddings = embed_spacings_in_position_embeddings(
                    absolute_position_embeddings, spacings
                )
                # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

            embeddings = embeddings + absolute_position_embeddings
            # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

        embeddings = rearrange(embeddings, "b d nz ny nx -> b (nz ny nx) d")

        return embeddings

In [26]:
test_config = {
    "patch_size": (8, 16, 16),
    "in_channels": 1,
    "dim": 768,
    "use_absolute_position_embeddings": True,
    "learnable_absolute_position_embeddings": False,
    "embed_spacing_info": False,
    "image_size": (32, 512, 512),
}

test = ViT3DEmbeddings(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 512, 512),
    torch.randn(2, 3),
)
display(o.shape)


ViT3DEmbeddings(
  (patch_embeddings): ViT3DPatchEmbeddings(
    (patch_embeddings): Conv3d(1, 768, kernel_size=(8, 16, 16), stride=(8, 16, 16))
  )
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

torch.Size([2, 4096, 768])

In [27]:
e = get_3d_position_embeddings(6, (2, 2, 2), (1, 1, 1))
display(e.shape)

print("Distance of (0, 0, 0) with other coords")
for i in range(2):
    for j in range(2):
        for k in range(2):
            print((i, j, k), np.linalg.norm(e[0, :, 0, 0, 0] - e[0, :, i, j, k]))

torch.Size([1, 6, 2, 2, 2])

Distance of (0, 0, 0) with other coords
(0, 0, 0) 0.0
(0, 0, 1) 0.95885104
(0, 1, 0) 0.95885104
(0, 1, 1) 1.3560202
(1, 0, 0) 0.95885104
(1, 0, 1) 1.3560202
(1, 1, 0) 1.3560202
(1, 1, 1) 1.6607786


# Models

In [28]:
# | export


class ViT3DModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config):
        super().__init__()

        self.embeddings = ViT3DEmbeddings(config)
        self.pos_drop = nn.Dropout(config.get("drop_prob", 0.0))
        self.num_class_tokens = config["num_class_tokens"]
        if self.num_class_tokens > 0:
            self.class_tokens = nn.Parameter(torch.randn(1, config["num_class_tokens"], config["dim"]))
        self.encoder = ViT3DEncoder(config)

    def forward(
        self,
        pixel_values: torch.Tensor,
        spacings: torch.Tensor,
        mask_patches: torch.Tensor = None,
        mask_token: torch.Tensor = None,
    ):
        # pixel_values: (b, c, z, y, x)
        # spacings: (b, 3)
        # mask_patches: (num_tokens_z, num_tokens_y, num_tokens_x)

        embeddings = self.embeddings(pixel_values, spacings, mask_patches, mask_token)
        embeddings = self.pos_drop(embeddings)
        # (b, num_tokens, dim)

        class_tokens = None
        if self.num_class_tokens > 0:
            class_tokens = repeat(self.class_tokens, "1 n d -> b n d", b=embeddings.shape[0])
            embeddings = torch.cat([class_tokens, embeddings], dim=1)
            # (b, num_tokens + 1, dim)

        encoded, layer_outputs = self.encoder(embeddings)
        # encoded: (b, num_tokens (+ 1), dim)
        # layer_outputs: list of (b, num_tokens (+ 1), dim)

        if self.num_class_tokens > 0:
            class_tokens = encoded[:, : self.num_class_tokens]
            encoded = encoded[:, self.num_class_tokens :]

        return class_tokens, encoded, layer_outputs

In [29]:
test_config = {
    "num_class_tokens": 2,
    "attn_drop_prob": 0.2,
    "dim": 768,
    "drop_prob": 0.2,
    "embed_spacing_info": False,
    "encoder_depth": 4,
    "image_size": (32, 512, 512),
    "in_channels": 1,
    "intermediate_ratio": 2,
    "layer_norm_eps": 1e-6,
    "learnable_absolute_position_embeddings": False,
    "mlp_drop_prob": 0.2,
    "num_heads": 4,
    "patch_size": (8, 16, 16),
    "proj_drop_prob": 0.2,
    "use_absolute_position_embeddings": True,
}

test = ViT3DModel(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 512, 512),
    torch.randn(2, 3),
)
display((o[0].shape, o[1].shape, [x.shape for x in o[2]]))


ViT3DModel(
  (embeddings): ViT3DEmbeddings(
    (patch_embeddings): ViT3DPatchEmbeddings(
      (patch_embeddings): Conv3d(1, 768, kernel_size=(8, 16, 16), stride=(8, 16, 16))
    )
    (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.2, inplace=False)
  (encoder): ViT3DEncoder(
    (layers): ModuleList(
      (0-3): 4 x ViT3DEncoderLayer(
        (mhsa): ViT3DMHSA(
          (W_q): Linear(in_features=768, out_features=768, bias=True)
          (W_k): Linear(in_features=768, out_features=768, bias=True)
          (W_v): Linear(in_features=768, out_features=768, bias=True)
          (attn_drop): Dropout(p=0.2, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.2, inplace=False)
        )
        (layernorm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): ViT3DLayerMLP(
          (dense1): Linear(in_features=768, out_features=1536, bias=True)
   


(
    torch.Size([2, 2, 768]),
    torch.Size([2, 4096, 768]),
    [
        torch.Size([2, 4098, 768]),
        torch.Size([2, 4098, 768]),
        torch.Size([2, 4098, 768]),
        torch.Size([2, 4098, 768])
    ]
)

# Masked Image Modeling

In [30]:
# | export


class ViT3DMIMDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.image_size = config["image_size"]
        self.in_channels = config["in_channels"]
        self.patch_size = config["patch_size"]

        dim = config["dim"]
        out_dim = np.prod(self.patch_size) * self.in_channels

        self.decoder = nn.Linear(dim, out_dim)

    def forward(self, encodings: torch.Tensor):
        # encodings: (b, num_tokens, dim)

        decoded = self.decoder(encodings)
        # (b, num_tokens, new_dim)

        decoded = rearrange(
            decoded,
            "b (nz ny nx) (c pz py px) -> b c (nz pz) (ny py) (nx px)",
            c=self.in_channels,
            pz=self.patch_size[0],
            py=self.patch_size[1],
            px=self.patch_size[2],
            nz=self.image_size[0] // self.patch_size[0],
            ny=self.image_size[1] // self.patch_size[1],
            nx=self.image_size[2] // self.patch_size[2],
        )
        # (b, c, z, y, x)

        return decoded

In [31]:
test_config = {
    "dim": 768,
    "image_size": (32, 512, 512),
    "in_channels": 1,
    "patch_size": (8, 16, 16),
}

test = ViT3DMIMDecoder(test_config)
display(test)
display(test(torch.randn(2, 4096, 768)).shape)


ViT3DMIMDecoder(
  (decoder): Linear(in_features=768, out_features=2048, bias=True)
)

torch.Size([2, 1, 32, 512, 512])

In [32]:
# | export


class ViT3DMIM(nn.Module):
    def __init__(self, config):
        super().__init__()

        assert config["num_class_tokens"] == 0, "MIM does not support class tokens"

        self.image_size = config["image_size"]
        self.patch_size = config["patch_size"]
        self.in_channels = config["in_channels"]
        self.mask_ratio = config["mask_ratio"]

        self.vit = ViT3DModel(config)
        self.decoder = ViT3DMIMDecoder(config)

        self.mask_token = nn.Parameter(torch.randn(1, config["dim"], 1, 1, 1))

    def mask_image(self, pixel_values: torch.Tensor):
        b = pixel_values.shape[0]

        mask_ratio = self.mask_ratio
        grid_size = tuple([size // patch for size, patch in zip(self.image_size, self.patch_size)])
        num_tokens = np.prod(grid_size)
        mask_patches = []
        for _ in range(b):
            _mask_patches = torch.zeros(num_tokens, dtype=torch.int8, device=pixel_values.device)
            _mask_patches[: int(mask_ratio * num_tokens)] = 1
            _mask_patches = _mask_patches[torch.randperm(num_tokens)]
            _mask_patches = rearrange(_mask_patches, "(z y x) -> z y x", z=grid_size[0], y=grid_size[1], x=grid_size[2])
            mask_patches.append(_mask_patches)
        mask_patches: torch.Tensor = torch.stack(mask_patches, dim=0)

        return mask_patches

In [33]:
# | export


class ViT3DSimMIM(ViT3DMIM, PyTorchModelHubMixin):
    def __init__(self, config):
        super().__init__(config)

    @staticmethod
    def loss_fn(pred: torch.Tensor, target: torch.Tensor, reduction="mean"):
        return nn.functional.l1_loss(pred, target, reduction=reduction)

    def forward(self, pixel_values: torch.Tensor, spacings: torch.Tensor):
        mask_patches = self.mask_image(pixel_values)

        _, encodings, _ = self.vit(pixel_values, spacings, mask_patches, self.mask_token)
        decoded = self.decoder(encodings)

        loss = self.loss_fn(decoded, pixel_values, reduction="none")
        mask = repeat(
            mask_patches,
            "b z y x -> b (z pz) (y py) (x px)",
            pz=self.patch_size[0],
            py=self.patch_size[1],
            px=self.patch_size[2],
        )
        loss = (loss * mask).sum() / ((mask.sum() + 1e-5) * self.in_channels)

        return decoded, loss, mask

In [34]:
test_config = {
    "num_class_tokens": 0,
    "attn_drop_prob": 0.2,
    "dim": 768,
    "drop_prob": 0.2,
    "embed_spacing_info": False,
    "encoder_depth": 4,
    "image_size": (32, 128, 128),
    "in_channels": 1,
    "intermediate_ratio": 2,
    "layer_norm_eps": 1e-6,
    "learnable_absolute_position_embeddings": False,
    "mlp_drop_prob": 0.2,
    "num_heads": 4,
    "patch_size": (8, 16, 16),
    "proj_drop_prob": 0.2,
    "use_absolute_position_embeddings": True,
    "mask_ratio": 0.8,
}

test = ViT3DSimMIM(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 128, 128),
    torch.randn(2, 3),
)
display((o[0].shape, o[1], o[2].shape))


ViT3DSimMIM(
  (vit): ViT3DModel(
    (embeddings): ViT3DEmbeddings(
      (patch_embeddings): ViT3DPatchEmbeddings(
        (patch_embeddings): Conv3d(1, 768, kernel_size=(8, 16, 16), stride=(8, 16, 16))
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.2, inplace=False)
    (encoder): ViT3DEncoder(
      (layers): ModuleList(
        (0-3): 4 x ViT3DEncoderLayer(
          (mhsa): ViT3DMHSA(
            (W_q): Linear(in_features=768, out_features=768, bias=True)
            (W_k): Linear(in_features=768, out_features=768, bias=True)
            (W_v): Linear(in_features=768, out_features=768, bias=True)
            (attn_drop): Dropout(p=0.2, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.2, inplace=False)
          )
          (layernorm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): ViT3DLayerMLP(
            (de


(
    torch.Size([2, 1, 32, 128, 128]),
    tensor(3.4644, grad_fn=<DivBackward0>),
    torch.Size([2, 32, 128, 128])
)

In [35]:
from neuro_utils.visualize import plot_scans

plot_scans([o[0][0, 0].detach(), o[0][0, 0].detach() * (1 - o[2][0])])

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

# Some more tests

### Overfitting tests

In [36]:
from tqdm.auto import tqdm

sample_spacings = torch.tensor([[1, 0.1, 0.1], [2, 0.2, 0.2], [3, 0.3, 0.3], [4, 0.4, 0.4], [5, 0.5, 0.5]])
sample_batch = torch.rand(3, 1, 16, 128, 128)
sample_config = {
    "num_class_tokens": 0,
    "attn_drop_prob": 0.2,
    "dim": 384,
    "drop_prob": 0.2,
    "embed_spacing_info": False,
    "encoder_depth": 4,
    "image_size": (16, 128, 128),
    "in_channels": 1,
    "intermediate_ratio": 2,
    "layer_norm_eps": 1e-6,
    "learnable_absolute_position_embeddings": False,
    "mlp_drop_prob": 0.2,
    "num_heads": 4,
    "patch_size": (8, 16, 16),
    "proj_drop_prob": 0.2,
    "use_absolute_position_embeddings": True,
    "mask_ratio": 0.8,
}

model = ViT3DSimMIM(sample_config)

sum(x.numel() for x in model.vit.parameters()), sum(x.numel() for x in model.decoder.parameters())

(5523072, 788480)

In [37]:
from neuro_utils.describe import describe_model

describe_model(model)

Total Parameters: 6,311,936
+---------------------------------------------------------+------------+
|                          Module                         | Parameters |
+---------------------------------------------------------+------------+
|                        mask_token                       |    384     |
| vit.embeddings.patch_embeddings.patch_embeddings.weight |  786,432   |
|  vit.embeddings.patch_embeddings.patch_embeddings.bias  |    384     |
|             vit.embeddings.layer_norm.weight            |    384     |
|              vit.embeddings.layer_norm.bias             |    384     |
|           vit.encoder.layers.0.mhsa.W_q.weight          |  147,456   |
|            vit.encoder.layers.0.mhsa.W_q.bias           |    384     |
|           vit.encoder.layers.0.mhsa.W_k.weight          |  147,456   |
|            vit.encoder.layers.0.mhsa.W_k.bias           |    384     |
|           vit.encoder.layers.0.mhsa.W_v.weight          |  147,456   |
|            vit.encode

In [38]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [39]:
sample_batch = sample_batch.cuda()
sample_spacings = sample_spacings.cuda()
model = model.cuda()

In [40]:
for i in tqdm(range(200)):
    optimizer.zero_grad()
    output = model(sample_batch, sample_spacings)
    print(f"Loss: {output[1]:f}\tLR: {scheduler.get_last_lr()[0]:f}")
    output[1].backward()
    optimizer.step()
    scheduler.step()

  0%|          | 0/200 [00:00<?, ?it/s]

Loss: 4.829837	LR: 0.500000
Loss: 5.426243	LR: 0.500000
Loss: 5.516611	LR: 0.500000
Loss: 4.618600	LR: 0.500000
Loss: 3.332942	LR: 0.500000
Loss: 2.991180	LR: 0.450000
Loss: 2.600994	LR: 0.450000
Loss: 2.489664	LR: 0.450000
Loss: 2.448338	LR: 0.450000
Loss: 2.410547	LR: 0.450000
Loss: 2.368401	LR: 0.405000
Loss: 2.287184	LR: 0.405000
Loss: 2.251144	LR: 0.405000
Loss: 2.222031	LR: 0.405000
Loss: 2.217494	LR: 0.405000
Loss: 2.206557	LR: 0.364500
Loss: 2.192892	LR: 0.364500
Loss: 2.175776	LR: 0.364500
Loss: 2.169362	LR: 0.364500
Loss: 2.159019	LR: 0.364500
Loss: 2.159636	LR: 0.328050
Loss: 2.127375	LR: 0.328050
Loss: 2.118558	LR: 0.328050
Loss: 2.101517	LR: 0.328050
Loss: 2.107554	LR: 0.328050
Loss: 2.091869	LR: 0.295245
Loss: 2.085718	LR: 0.295245
Loss: 2.080989	LR: 0.295245
Loss: 2.073884	LR: 0.295245
Loss: 2.071027	LR: 0.295245
Loss: 2.072474	LR: 0.265721
Loss: 2.064900	LR: 0.265721
Loss: 2.062473	LR: 0.265721
Loss: 2.056371	LR: 0.265721
Loss: 2.060110	LR: 0.265721
Loss: 2.045528	LR: 0

In [41]:
for name, param in model.named_parameters():
    if param.grad is None:
        print(name)

# nbdev

In [42]:
!nbdev_export